# Recoinize

# 1. Setup

## 1.1. Pull code from GitHub

In [4]:
!git fetch --all
 
# Switch to the "live-edit" branch, ignore local changes, and pull the latest changes
!git checkout live-edit
!git reset --hard
!git pull

Fetching origin
M	main.ipynb
Branch 'live-edit' set up to track remote branch 'live-edit' from 'origin'.
Switched to a new branch 'live-edit'
HEAD is now at b3a6e6e Set up live edit system for paperspace gradient runs (#1)
Already up to date.


ss


In [1]:
!nvidia-smi

Thu Jan 19 10:07:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        Off  | 00000000:00:05.0 Off |                  Off |
| 26%   21C    P8     5W / 180W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1.2. Define constants

Settings are in the top of the notebook, so you can easily change them.

In [2]:
NB_OF_IMAGES = 1000
DATA_FOLDER = 'data'
OUT_FOLDER = 'out'
FOLDER_NAMES = ['positive', 'negative', 'anchor']

## 1.3. Install dependencies

In [ ]:
from helpers.install import install_requirements

install_requirements()

## 1.4. Import dependencies

In [5]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

# Import tensorflow dependencies - Functional API
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential, load_model

2023-01-19 10:10:33.818949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 10:10:34.708297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist-packages/cv2/../../lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-01-19 10:10:34.708435: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.9/dist

## 1.5 Set GPU Growth

In [5]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 2. Get data

## 2.1. Define folder structure

This folder names are defined in our script (cf. this [repo](https://github.com/photonsquid/Recoinize-generator)) that generates the dataset.
Then it is uploaded to our [Hugging Face dataset](https://huggingface.co/datasets/photonsquid/coins-euro).

In [13]:
FOLDER_PATHS = {folder: os.path.join(DATA_FOLDER, folder) for folder in FOLDER_NAMES}

# Create folders if they don't exist
for folder in FOLDER_PATHS.values():
    if not os.path.exists(folder):
        os.makedirs(folder)

This is what the `FOLDER_PATHS` variable looks like:

```python
FOLDER_PATHS = {
    "positive": "data/positive",
    "negative": "data/negative",
    "anchor": "data/anchor",
}
```

At this point, the folders exist, but they are empty.

## 2.2. Download data

If it is not already done, download data.

In [2]:
# TODO: implement it with huggin face library

Now we have downloaded all the dataset, and we should have the following folder structure:

```bash
data
├── negative
│   ├── va_2euro_01.jpg
│   ├── va_2euro_02.jpg
│   ├── va_2euro_03.jpg
│   ...
├── positive
│   ├── at_2euro_00.jpg
│   ├── at_2euro_01.jpg
│   ├── at_2euro_02.jpg
│   ...
└── anchor
    ├── fr_2euro_00.jpg
    ├── fr_2euro_01.jpg
    ├── fr_2euro_02.jpg
    ...
```

## 2.3. Load data

### 2.3.1 Load data into TensorFlow Datasets.

Take `n` images from each folder in the dataset.

In [ ]:
images = {tf.data.Dataset.list_files(FOLDER_PATHS[folder]+'\*.jpg').take(NB_OF_IMAGES) for folder in FOLDER_NAMES}

## 3. Build the model

Let's say that here we have a clean Tensorflow dataset:
    
- `train_ds` is the training dataset
- `test_ds` 
- `val_ds` is the validation dataset
  
Each dataset is a `tf.data.Dataset` object.

we also have Anchor, Positive and Negative images.

### 3.1 The embedding model